In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymongo import MongoClient
import time

In [2]:
DEBUG=True
USE_DATABASE=True

In [3]:
if USE_DATABASE:
    client = MongoClient("mongodb://192.168.1.200:27017")
    db = client["test"] # Connect to db 'test'
    
    # Get data from matchapps collection
    matchapps_collection = db["matchapps"]
    match_df = pd.json_normalize(matchapps_collection.find())

    # Save a backup for later use
    match_df.to_csv(f"matchapps_backup_{time.time()}.csv")

    # Get data from superapps collection
    superapps_collection = db["superapps"]
    super_df = pd.json_normalize(superapps_collection.find())
    super_df.to_csv(f"superapps_backup_{time.time()}.csv")
else:
    match_df = pd.read_csv("matchapps.csv")
    super_df = pd.read_csv("superapps.csv")

match_df.sort_values("metadata.matchNumber", inplace=True)
match_df

,_id,leftStartingZone,climb,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,startingZone.start1,startingZone.start2,...,autoAlgae.netRobot,autoAlgae.processor,autoAlgae.remove,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4,teleAlgae.netRobot,teleAlgae.processor,teleAlgae.remove
0,67c344ea114fd247353f8e0f,True,shallow,0,Kiinon2,1,3255.0,blue_1,True,False,...,0,0,0,0,0,0,0,1,0,0
25,67c352cd114fd247353f8ec8,True,none,0,Nathan,1,2375.0,blue_3,False,False,...,0,0,0,0,1,3,0,0,0,1
52,67c35338114fd247353f8fc4,True,park,0,Christian,1,399.0,blue_2,False,True,...,0,0,0,0,0,0,0,0,0,0
13,67c352cd114fd247353f8e98,True,park,0,,1,1388.0,red_2,False,False,...,0,0,0,0,0,1,8,0,0,0
37,67c352e2114fd247353f8ef8,True,deep,0,Chiamaka,1,2543.0,red_1,False,True,...,0,0,0,0,0,0,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,67c4ab98c08944667710f99b,True,park,0,Zach,71,2543.0,blue_2,False,False,...,0,0,1,0,2,0,2,0,0,0
408,67c4ab8fc08944667710f979,True,failed,0,Joaquin,71,4201.0,red_1,False,True,...,0,0,0,0,0,3,7,0,0,1
399,67c4ab81c08944667710f953,True,park,0,Mila,71,8119.0,red_2,False,False,...,0,0,0,0,2,3,0,0,1,3
426,67c4abb1c08944667710fa2b,True,failed,0,Soso,71,8891.0,blue_1,False,False,...,0,0,0,0,1,0,2,2,0,2


# Data Cleaning

Things I've noticed:
- Robots scoring on reef while not being marked for leave
- Crazy amounts scored in some matches
- Duplicate matches
- Database not cleared ahead of time - some dummy data left over

In [4]:
# Robots scoring on reef while not being marked for leave
match_df[
    (~match_df["leftStartingZone"]) & (
    (match_df["autoAlgae.netRobot"] > 0) |
    (match_df["autoAlgae.processor"] > 0) |
    (match_df["autoAlgae.remove"] > 0) |
    (match_df["autoCoral.L1"] > 0) |
    (match_df["autoCoral.L2"] > 0) |
    (match_df["autoCoral.L3"] > 0) |
    (match_df["autoCoral.L4"] > 0))
][
    ["metadata.robotTeam", "metadata.matchNumber", "leftStartingZone"] +
    ["autoAlgae.netRobot", "autoAlgae.processor", "autoAlgae.remove"] +
    ["autoCoral.L{}".format(x) for x in range(1, 5)]
]

,metadata.robotTeam,metadata.matchNumber,leftStartingZone,autoAlgae.netRobot,autoAlgae.processor,autoAlgae.remove,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4
4,610.0,1,False,0,0,0,0,1,0,0
7,2102.0,3,False,0,0,1,1,0,0,0
29,2584.0,5,False,1,0,0,0,0,0,1
9,8015.0,6,False,0,0,0,0,0,0,1
66,1159.0,7,False,0,0,0,1,0,0,0
117,3598.0,18,False,0,0,0,0,0,0,1
139,3863.0,23,False,0,0,1,0,0,1,0
151,8015.0,27,False,0,0,0,0,0,1,4
201,3255.0,35,False,0,0,0,0,0,0,1
225,2584.0,36,False,1,0,0,0,0,0,1


In [5]:
# Indexes below all have leave marked as false and should be true,
# manually verified by watching match data
incorrect_leave_indexes = [4, 7, 9, 29, 66, 117, 139, 151, 201, 225, 226, 227, 264]

for i in incorrect_leave_indexes:
    match_df.loc[i, "leftStartingZone"] = True

match_df[match_df.index.isin(incorrect_leave_indexes)]

,_id,leftStartingZone,climb,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,startingZone.start1,startingZone.start2,...,autoAlgae.netRobot,autoAlgae.processor,autoAlgae.remove,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4,teleAlgae.netRobot,teleAlgae.processor,teleAlgae.remove
4,67c34e9b114fd247353f8e74,True,none,0,Carlos,1,610.0,red_3,True,False,...,0,0,0,0,1,4,5,0,0,2
7,67c34e9b114fd247353f8e7d,True,park,0,Carlos,3,2102.0,red_3,True,False,...,0,0,1,0,0,0,0,0,2,0
29,67c352cd114fd247353f8ed0,True,none,0,Nathan,5,2584.0,blue_3,False,True,...,1,0,0,0,1,2,2,0,0,2
9,67c34e9b114fd247353f8e81,True,none,0,Carlos,6,8015.0,red_3,False,True,...,0,0,0,0,0,0,6,0,0,0
66,67c366f2114fd247353f908d,True,shallow,0,Kkinnon2,7,1159.0,blue_1,False,True,...,0,0,0,0,0,0,0,0,0,0
117,67c36a6c114fd247353f91fb,True,deep,0,David G.,18,3598.0,red_3,False,True,...,0,0,0,0,0,0,2,0,0,0
139,67c3819d3822051181ced053,True,failed,0,David G.,23,3863.0,red_3,False,False,...,0,0,1,0,1,5,0,0,1,2
151,67c383273822051181ced07b,True,park,0,Josie,27,8015.0,red_1,False,False,...,0,0,0,0,0,0,0,0,0,0
201,67c39027a522b4c28eabe134,True,failed,0,Stephen,35,3255.0,red_2,False,False,...,0,0,0,0,0,4,7,0,0,3
225,67c3a03ba522b4c28eabe1f3,True,deep,0,Josie,36,2584.0,red_1,False,True,...,1,0,0,0,0,0,1,1,0,2


In [6]:
# Robots scoring crazy amounts, imposing reasonable defaults
match_df[
    (match_df["autoCoral.L1"] > 5) |
    (match_df["autoCoral.L2"] > 5) |
    (match_df["autoCoral.L3"] > 5) |
    (match_df["autoCoral.L4"] > 5) |
    (match_df["teleCoral.L1"] > 10) |
    (match_df["teleCoral.L2"] > 10) |
    (match_df["teleCoral.L3"] > 10) |
    (match_df["teleCoral.L4"] > 10)
][
    ["metadata.robotTeam", "metadata.matchNumber"] +
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)]
]


,metadata.robotTeam,metadata.matchNumber,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4
335,3863.0,43,0,0,0,0,3,0,22,0


In [7]:
# Index 335 - update values to:
# 4 L1
# 8 L3

match_df.loc[335, "teleCoral.L1"] = 4
match_df.loc[335, "teleCoral.L3"] = 8

match_df[match_df.index == 335][
    ["metadata.robotTeam", "metadata.matchNumber"] +
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)]
]

,metadata.robotTeam,metadata.matchNumber,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4
335,3863.0,43,0,0,0,0,4,0,8,0


In [8]:
# Find duplicate matches
# match_df.groupby(["metadata.robotTeam", "metadata.matchNumber"]).count() != 1
# match_df.groupby("metadata.matchNumber").count() != 6

In [30]:
# Find teams that aren't at this tournament
match_df[match_df["metadata.robotTeam"] == 6904]

,_id,leftStartingZone,climb,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,startingZone.start1,startingZone.start2,...,autoScore,teleScore,climb.park,climb.shallow,climb.deep,endgameScore,totalScore,metadata.robotColorIsBlue,autoScoreRightReef,autoScoreLeftReef
342,67c48f7ec08944667710f7d8,False,deep,0,Vanessa,21,6904.0,red_1,NaN,NaN,...,0,27,False,False,True,12,39,False,False,False


# Adding New Fields

In [9]:
# Grab a random sample match
DEBUG and match_df[(match_df["metadata.robotTeam"] == 4201) & (match_df["metadata.matchNumber"] == 8)]

,_id,leftStartingZone,climb,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,startingZone.start1,startingZone.start2,...,autoAlgae.netRobot,autoAlgae.processor,autoAlgae.remove,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4,teleAlgae.netRobot,teleAlgae.processor,teleAlgae.remove
59,67c35339114fd247353f8fdd,True,park,0,Christian,8,4201.0,blue_2,False,True,...,0,0,0,0,2,5,4,0,0,0


In [10]:
# Add columns for auto, tele, and endgame score
match_df["autoScore"] = (
    match_df["leftStartingZone"] * 3 +
    match_df["autoCoral.L1"] * 3 +
    match_df["autoCoral.L2"] * 4 +
    match_df["autoCoral.L3"] * 6 +
    match_df["autoCoral.L4"] * 7 +
    match_df["autoAlgae.processor"] * 6 +
    match_df["autoAlgae.netRobot"] * 4
)

match_df["teleScore"] = (
    match_df["teleCoral.L1"] * 2 +
    match_df["teleCoral.L2"] * 3 +
    match_df["teleCoral.L3"] * 4 +
    match_df["teleCoral.L4"] * 5 +
    match_df["teleAlgae.processor"] * 6 +
    match_df["teleAlgae.netRobot"] * 4
)

match_df["climb.park"] = match_df["climb"] == "park"
match_df["climb.shallow"] = match_df["climb"] == "shallow"
match_df["climb.deep"] = match_df["climb"] == "deep"

match_df["endgameScore"] = (
    match_df["climb.park"] * 2 +
    match_df["climb.shallow"] * 6 +
    match_df["climb.deep"] * 12
)

match_df["totalScore"] = (
    match_df["autoScore"] +
    match_df["teleScore"] +
    match_df["endgameScore"]
)


In [11]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["autoCoral.L1", "autoCoral.L2", "autoCoral.L3", "autoCoral.L4", "autoAlgae.processor", "autoAlgae.netRobot", "autoScore"]]

,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4,autoAlgae.processor,autoAlgae.netRobot,autoScore
0,0,0,0,0,0,0,3
94,0,0,0,2,0,0,17
109,1,0,0,0,0,0,6
142,0,0,0,1,0,0,10
201,0,0,0,1,0,0,10
213,0,0,0,1,0,0,10
277,0,0,0,2,0,0,17
345,0,1,2,1,0,1,30
370,0,0,0,2,0,0,17
402,0,0,1,1,0,0,13


In [12]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["leftStartingZone", "teleCoral.L1", "teleCoral.L2", "teleCoral.L3", "teleCoral.L4", "teleAlgae.processor", "teleAlgae.netRobot", "teleScore"]]

,leftStartingZone,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4,teleAlgae.processor,teleAlgae.netRobot,teleScore
0,True,0,0,0,0,0,1,4
94,True,0,5,2,3,0,1,42
109,True,2,0,0,4,0,2,32
142,True,0,0,0,1,0,5,25
201,True,0,0,4,7,0,0,51
213,True,0,5,4,3,0,1,50
277,True,0,0,5,6,0,4,66
345,True,2,3,2,5,0,3,58
370,True,0,2,5,2,0,2,44
402,False,0,1,6,5,0,3,64


In [13]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["climb.park", "climb.shallow", "climb.deep", "endgameScore"]]

,climb.park,climb.shallow,climb.deep,endgameScore
0,False,True,False,6
94,False,False,True,12
109,False,False,True,12
142,False,False,True,12
201,False,False,False,0
213,False,False,True,12
277,False,False,False,0
345,False,True,False,6
370,False,False,True,12
402,True,False,False,2


In [14]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["autoScore", "teleScore", "endgameScore", "totalScore"]]

,autoScore,teleScore,endgameScore,totalScore
0,3,4,6,13
94,17,42,12,71
109,6,32,12,50
142,10,25,12,47
201,10,51,0,61
213,10,50,12,72
277,17,66,0,83
345,30,58,6,94
370,17,44,12,73
402,13,64,2,79


In [15]:
# Add columns for whether teams have scored on the side of the reef in auto
match_df["metadata.robotColorIsBlue"] = (
    (match_df["metadata.robotPosition"] == "blue_1") |
    (match_df["metadata.robotPosition"] == "blue_2") |
    (match_df["metadata.robotPosition"] == "blue_3")
)

# Calculate if a team scored on the right side of the reef, from the perspective of the barge
match_df["autoScoreRightReef"] = (
    match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit1"] | match_df["placement.deposit6"]) |
    ~match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit3"] | match_df["placement.deposit4"])
)

# Calculate if a team scored on the left side of the reef, from the perspective of the barge
match_df["autoScoreLeftReef"] = (
    match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit3"] | match_df["placement.deposit4"]) |
    ~match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit1"] | match_df["placement.deposit6"])
)

In [16]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["autoScoreRightReef"] | match_df["autoScoreLeftReef"]][["metadata.matchNumber", "metadata.robotTeam", "metadata.robotPosition", "placement.deposit1", "placement.deposit2", "placement.deposit3", "placement.deposit4", "placement.deposit5", "placement.deposit6", "autoScoreRightReef", "autoScoreLeftReef"]]

,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,placement.deposit1,placement.deposit2,placement.deposit3,placement.deposit4,placement.deposit5,placement.deposit6,autoScoreRightReef,autoScoreLeftReef
0,1,3255.0,blue_1,False,False,False,False,False,True,True,False
4,1,610.0,red_3,False,False,False,False,False,True,False,True
7,3,2102.0,red_3,False,False,False,False,False,True,False,True
19,7,5515.0,red_2,False,False,False,False,False,True,False,True
11,8,5817.0,red_3,False,False,False,True,False,False,True,False
12,9,702.0,red_3,True,False,False,False,False,False,False,True
22,10,4414.0,red_2,False,False,False,False,False,True,False,True
49,10,10147.0,red_3,False,False,False,True,False,False,True,False
35,11,9599.0,blue_3,False,False,True,False,False,False,False,True
47,11,498.0,red_1,False,False,True,False,False,False,True,False


In [17]:
team_df = pd.DataFrame()

In [18]:
# For each of the columns below, find only the average
AVG_COLUMNS = (
    ["startingZone.start{}".format(x) for x in range(1, 4)] +
    ["pickupLocation.source{}".format(x) for x in range(1, 3)] +
    ["pickupLocation.ground{}".format(x) for x in range(1, 4)] +
    ["placement.deposit{}".format(x) for x in range(1, 7)] +
    ["climb.park", "climb.shallow", "climb.deep", "autoScoreRightReef", "autoScoreLeftReef"]
)

for c in AVG_COLUMNS:
    team_df[f"{c}.avg"] = match_df.groupby("metadata.robotTeam")[c].mean()

In [19]:
# For each of the columns below, find statistical metrics for each (min, max, avg, std)
STAT_COLUMNS = (
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)] +
    ["autoAlgae.netRobot", "autoAlgae.processor", "autoAlgae.remove"] +
    ["teleAlgae.netRobot", "teleAlgae.processor", "teleAlgae.remove"] +
    ["autoScore", "teleScore", "endgameScore", "totalScore"]
)

for c in STAT_COLUMNS:
    team_df[f"{c}.min"] = match_df.groupby("metadata.robotTeam")[c].min()
    team_df[f"{c}.max"] = match_df.groupby("metadata.robotTeam")[c].max()
    team_df[f"{c}.avg"] = match_df.groupby("metadata.robotTeam")[c].mean()
    team_df[f"{c}.std"] = match_df.groupby("metadata.robotTeam")[c].std()

    # We also want to calculate these metrics for different variations of matches being dropped.
    for n in range(1, 3):
        # Start by removing the top and bottom `n` matches for a team by finding
        # the indexes of the matches we want to keep
        indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.get_level_values(1)
        filtered_df = match_df[match_df.index.isin(indexes_to_keep)]

        # With our new filtered dataframe, compute statistic metrics
        team_df[f"{c}.min.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].min()
        team_df[f"{c}.max.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].max()
        team_df[f"{c}.avg.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].mean()
        team_df[f"{c}.std.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].std()

/tmp/ipykernel_313928/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.get_level_values(1)
/tmp/ipykernel_313928/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.ge

In [20]:
DEBUG and team_df[team_df.index == 4201][["autoCoral.L4.avg", "autoCoral.L4.avg.drop1", "autoCoral.L4.avg.drop2"]]

,autoCoral.L4.avg,autoCoral.L4.avg.drop1,autoCoral.L4.avg.drop2
metadata.robotTeam,,,
4201.0,0.5,0.5,0.5


In [21]:
DEBUG and match_df[match_df["metadata.robotTeam"] == 4201][["metadata.robotTeam", "autoCoral.L4"]]

,metadata.robotTeam,autoCoral.L4
59,4201.0,0
95,4201.0,1
134,4201.0,0
203,4201.0,1
173,4201.0,1
269,4201.0,1
329,4201.0,0
344,4201.0,0
364,4201.0,0
408,4201.0,1


In [22]:
team_df["fouls.avg"] = super_df[[
    "fouls.insideRobot",
    "fouls.protectedZone",
    "fouls.pinning",
    "fouls.multiplePieces",
    "fouls.cageFoul",
    "fouls.other"
]].sum(axis=1).groupby(super_df["metadata.robotTeam"]).mean()

/tmp/ipykernel_313928/1034481741.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["fouls.avg"] = super_df[[


In [23]:
DEBUG and team_df["fouls.avg"]

metadata.robotTeam
4.0        0.111111
114.0      0.111111
399.0      0.000000
498.0      0.100000
610.0      0.111111
702.0      0.000000
988.0      0.090909
991.0      0.222222
1159.0     0.222222
1388.0     0.000000
1452.0     0.100000
2102.0     0.100000
2375.0     0.181818
2485.0     0.400000
2543.0     0.000000
2584.0     0.300000
2658.0     0.444444
2839.0     0.400000
3255.0     0.333333
3598.0     0.111111
3647.0     1.222222
3759.0     0.000000
3863.0     0.000000
3881.0     0.300000
4201.0     0.000000
4400.0     0.555556
4414.0     0.100000
5025.0     0.000000
5137.0     0.000000
5515.0     1.200000
5817.0     0.000000
6560.0     0.900000
6657.0     0.222222
6658.0     0.444444
6766.0     0.181818
6885.0     0.000000
6904.0          NaN
8015.0     0.000000
8119.0     0.000000
8768.0     0.111111
8891.0     0.090909
9452.0     0.000000
9534.0     0.555556
9599.0     0.000000
9772.0     0.600000
10120.0    0.000000
10147.0    0.000000
Name: fouls.avg, dtype: float64

In [24]:
team_df["break.avg"] = super_df[[
    "break.mechanismDmg",
    "break.batteryFall",
    "break.commsFail",
    "break.bumperFall"
]].sum(axis=1).groupby(super_df["metadata.robotTeam"]).mean()

/tmp/ipykernel_313928/634828927.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["break.avg"] = super_df[[


In [25]:
DEBUG and team_df["break.avg"]

metadata.robotTeam
4.0        0.000000
114.0      0.000000
399.0      0.000000
498.0      0.000000
610.0      0.000000
702.0      0.000000
988.0      0.090909
991.0      0.222222
1159.0     0.000000
1388.0     0.000000
1452.0     0.000000
2102.0     0.000000
2375.0     0.090909
2485.0     0.100000
2543.0     0.000000
2584.0     0.000000
2658.0     0.222222
2839.0     0.000000
3255.0     0.000000
3598.0     0.000000
3647.0     0.000000
3759.0     0.111111
3863.0     0.000000
3881.0     0.000000
4201.0     0.181818
4400.0     0.000000
4414.0     0.000000
5025.0     0.200000
5137.0     0.000000
5515.0     0.000000
5817.0     0.000000
6560.0     0.000000
6657.0     0.111111
6658.0     0.555556
6766.0     0.000000
6885.0     0.300000
6904.0          NaN
8015.0     0.000000
8119.0     0.000000
8768.0     0.000000
8891.0     0.090909
9452.0     0.000000
9534.0     0.000000
9599.0     0.000000
9772.0     0.400000
10120.0    0.000000
10147.0    0.000000
Name: break.avg, dtype: float64

In [26]:
# All copied from last year, didn't actually check if this works
def safeDivide(a, b):
    return 0 if b == 0 else a / b

team_list = match_df["metadata.robotTeam"].unique().tolist()
team_list = [team for team in team_list if not np.isnan(team)]

team_df["Percent of Matches with No Defense"] = None
team_df["Percent of Matches with Some Defense"] = None
team_df["Percent of Matches with Full Defense"] = None
team_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = super_df.loc[super_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    team_df.at[team, "Percent of Matches with No Defense"] = safeDivide(no_defense_count, defense_total_count)
    team_df.at[team, "Percent of Matches with Some Defense"] = safeDivide(some_defense_count, defense_total_count)
    team_df.at[team, "Percent of Matches with Full Defense"] = safeDivide(full_defense_count, defense_total_count)
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    team_df.at[team, "Main Defense Type"] = main_defense_type

/tmp/ipykernel_313928/1343473457.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["Percent of Matches with No Defense"] = None
/tmp/ipykernel_313928/1343473457.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["Percent of Matches with Some Defense"] = None
/tmp/ipykernel_313928/1343473457.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

In [27]:
DEBUG and team_df

,startingZone.start1.avg,startingZone.start2.avg,startingZone.start3.avg,pickupLocation.source1.avg,pickupLocation.source2.avg,pickupLocation.ground1.avg,pickupLocation.ground2.avg,pickupLocation.ground3.avg,placement.deposit1.avg,placement.deposit2.avg,...,totalScore.min.drop2,totalScore.max.drop2,totalScore.avg.drop2,totalScore.std.drop2,fouls.avg,break.avg,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
4.0,0.333333,0.111111,0.444444,0.111111,0.0,0.000000,0.111111,0.0,0.000000,0.000000,...,19.0,27.0,23.200000,3.563706,0.111111,0.000000,1.0,0.0,0.0,No Defense
114.0,0.111111,0.555556,0.111111,0.111111,0.0,0.000000,0.111111,0.0,0.000000,0.000000,...,9.0,19.0,12.400000,3.911521,0.111111,0.000000,0.777778,0.222222,0.0,No Defense
399.0,0.222222,0.555556,0.222222,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.111111,...,28.0,40.0,34.600000,4.774935,0.000000,0.000000,0.888889,0.111111,0.0,No Defense
498.0,0.333333,0.0,0.555556,0.111111,0.0,0.000000,0.000000,0.0,0.222222,0.000000,...,28.0,50.0,39.600000,10.454664,0.100000,0.000000,0.8,0.2,0.0,No Defense
610.0,0.444444,0.111111,0.444444,0.111111,0.222222,0.000000,0.000000,0.0,0.000000,0.000000,...,38.0,51.0,44.400000,6.188699,0.111111,0.000000,0.777778,0.111111,0.111111,No Defense
702.0,0.333333,0.111111,0.444444,0.111111,0.111111,0.000000,0.000000,0.222222,0.222222,0.111111,...,26.0,46.0,33.000000,9.027735,0.000000,0.000000,0.888889,0.111111,0.0,No Defense
988.0,0.3,0.2,0.5,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,3.0,5.0,4.333333,1.032796,0.090909,0.090909,0.454545,0.090909,0.454545,No Defense Full Defense
991.0,0.333333,0.111111,0.111111,0.0,0.0,0.000000,0.111111,0.222222,0.000000,0.000000,...,0.0,12.0,6.000000,4.898979,0.222222,0.222222,0.888889,0.0,0.111111,No Defense
1159.0,0.0,0.666667,0.111111,0.0,0.0,0.111111,0.000000,0.0,0.000000,0.222222,...,5.0,7.0,5.800000,1.095445,0.222222,0.000000,0.777778,0.111111,0.111111,No Defense


In [28]:
team_df.to_csv("caph_2025_sun.csv")

# Compare to TBA Data

In [35]:
EVENT_KEY = "2025caph"

In [37]:
import requests

params = [

]

url = f"https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/matches"

resp = requests.get(url=url, params=params)
data = resp.json() # Check the JSON Response Content documentation below


In [39]:
tba_df = pd.json_normalize(data)

In [49]:
for i in tba_df.index:
    print(tba_df.columns[i])

actual_time
comp_level
event_key
key
match_number
post_result_time
predicted_time
set_number
time
videos
winning_alliance
alliances.blue.dq_team_keys
alliances.blue.score
alliances.blue.surrogate_team_keys
alliances.blue.team_keys
alliances.red.dq_team_keys
alliances.red.score
alliances.red.surrogate_team_keys
alliances.red.team_keys
score_breakdown.blue.adjustPoints
score_breakdown.blue.algaePoints
score_breakdown.blue.autoBonusAchieved
score_breakdown.blue.autoCoralCount
score_breakdown.blue.autoCoralPoints
score_breakdown.blue.autoLineRobot1
score_breakdown.blue.autoLineRobot2
score_breakdown.blue.autoLineRobot3
score_breakdown.blue.autoMobilityPoints
score_breakdown.blue.autoPoints
score_breakdown.blue.autoReef.botRow.nodeA
score_breakdown.blue.autoReef.botRow.nodeB
score_breakdown.blue.autoReef.botRow.nodeC
score_breakdown.blue.autoReef.botRow.nodeD
score_breakdown.blue.autoReef.botRow.nodeE
score_breakdown.blue.autoReef.botRow.nodeF
score_breakdown.blue.autoReef.botRow.nodeG
scor